### In this notebook, I assessed the reconstruction method by comparing the CNAs of internal nodes of the ground truth tree and the reconstructed summary tree

In [30]:
import pandas as pd
import numpy as np
import dendropy
from collections import defaultdict

In [31]:
### Let's define at the ground true tree and states

with open('gt_unscaled.nw') as f:
    lines = f.read()
true_tree = lines[:-2]
true_tree += ";"
true_tree = dendropy.Tree.get_from_string(true_tree, "newick")

true_states = pd.read_table("all_states.txt")

f.close()

In [32]:
true_tree.print_plot()

                                                                                                      /------- leaf36 
                                               /------------------------------------------------------+               
                                               |                                                      \------- leaf37 
                                               |                                                                      
                                               |                                              /--------------- leaf90 
                                               |                                      /-------+                       
                                               |                                      |       |       /------- leaf96 
                                               |                                      |       \-------+               
                                               |

In [33]:
### Let's define the summary tree and states

with open('Base_asr.tre') as f:
    lines = f.readlines()
fake_treeS = lines[2:]

f.close()

In [34]:
fake_treeS[0]

### The state goes after the leaf label. Example: leaf100<1>_2 here, 2 is the state of leaf100

'  TREE chr1_1_594516 = ((((((((((((((((leaf100<1>_2:1.56985,leaf98<0>_2:1.56985)<7>_2:0.26467,leaf99<6>_2:1.83452)<11>_2:0.0457063,leaf86<10>_2:1.88023)<15>_2:1.7592,leaf101<14>_2:3.63943)<19>_2:4.78312,(((leaf90<5>_2:2.7086,leaf92<4>_2:2.7086)<9>_2:1.21957,(leaf96<3>_2:1.36745,leaf97<2>_2:1.36745)<8>_2:2.56071)<13>_2:0.115809,leaf93<12>_2:4.04398)<18>_2:4.37857)<25>_2:3.91937,leaf39<24>_2:12.3419)<37>_2:2.3093,(leaf36<23>_2:10.5611,(leaf77<17>_2:10.0701,leaf78<16>_2:10.0701)<22>_2:0.491029)<36>_2:4.09012)<51>_2:0.0573356,(leaf88<35>_2:6.04388,leaf89<34>_2:6.04388)<50>_2:8.66467)<61>_2:0.0346603,(leaf37<49>_2:2.88634,leaf63<48>_2:2.88634)<60>_2:11.8569)<69>_2:1.51629,leaf76<68>_2:16.2595)<77>_2:2.47984,leaf23<76>_2:18.7393)<85>_2:1.87635,leaf17<84>_2:20.6157)<93>_2:2.51755,leaf9<92>_2:23.1332)<97>_2:3.19982,((leaf11<83>_2:9.3938,leaf18<82>_2:9.3938)<91>_2:14.3685,leaf19<90>_2:23.7623)<96>_2:2.57072)<99>_2:9.07005,(((leaf13<81>_2:17.8579,(leaf80<75>_2:9.30363,leaf81<74>_2:9.30363)<80>_

In [9]:
### This piece of code compares the states of the true tree internal nodes with the states of the reconstructed trees

### Simple writeup
# foreach node in true_tree:
#     leaves <- the leaf nodes of the node
#     foreach gene summary_tree:
#         mrca <- the Most Recent Common Ancestor of leaves in the summary tree
#         compare the states of node and mrca and add to statistics    
###

### True tree nodes
nodes_labels = [i.label for i in true_tree.internal_nodes()[1:]]

output = []
count = 0
correct = 0
difference = defaultdict(int)

### Trees start at line 2
for i in range(2, len(fake_treeS)-2): #len(fake_treeS)-1
    print("Character: ", i*100/len(fake_treeS), "%")
    ### Looking at tree #i
    tree_i = fake_treeS[i].split(" = ")[1][:-1]
    tree_i_FamilyID = fake_treeS[i].split(" = ")[0][7:]
    fake_tree_i = dendropy.Tree.get_from_string(tree_i, "newick")
    
    ### Map from true_tree leaves labels to fake_tree leaves labels
    fake_tree_labels = [j.label for j in fake_tree_i.taxon_namespace]
    true_tree_labels = [j.label for j in true_tree.taxon_namespace]

    mapping = defaultdict(int)
    for k in true_tree_labels:
        for j in fake_tree_labels:
            if k in j:
                mapping[k] = j
    
    ### For each particular internal node in the true tree, find its leaves
    for node in true_tree.internal_nodes()[1:]:
        leaves = node.leaf_nodes()
        fake_leaves_labels = [mapping[i.taxon.label] for i in leaves]
        fake_node = fake_tree_i.mrca(taxon_labels=fake_leaves_labels)
        ### Check if they have the same leaves
        fake_leaves_labels.sort()
        all_leaves_fake_node = [i.taxon.label for i in fake_node.leaf_nodes()]
        all_leaves_fake_node.sort()
#!!!!   if all_leaves_fake_node == fake_leaves_labels:
            ### Compare
        arr = true_states[true_states["Family ID"] == tree_i_FamilyID].copy(deep=True)
        node_state = arr.reset_index().at[0, node.label]

        fake_node_label, fake_state = fake_node.label.split()
        fake_state = int(fake_state)

        if fake_state == node_state:
            correct += 1

        difference[node_state - fake_state] += 1     

        count += 1

Character:  0.012903225806451613 %
Character:  0.01935483870967742 %
Character:  0.025806451612903226 %
Character:  0.03225806451612903 %
Character:  0.03870967741935484 %
Character:  0.04516129032258064 %
Character:  0.05161290322580645 %
Character:  0.05806451612903226 %
Character:  0.06451612903225806 %
Character:  0.07096774193548387 %
Character:  0.07741935483870968 %
Character:  0.08387096774193549 %
Character:  0.09032258064516129 %
Character:  0.0967741935483871 %
Character:  0.1032258064516129 %
Character:  0.10967741935483871 %
Character:  0.11612903225806452 %
Character:  0.12258064516129032 %
Character:  0.12903225806451613 %
Character:  0.13548387096774195 %
Character:  0.14193548387096774 %
Character:  0.14838709677419354 %
Character:  0.15483870967741936 %
Character:  0.16129032258064516 %
Character:  0.16774193548387098 %
Character:  0.17419354838709677 %
Character:  0.18064516129032257 %
Character:  0.1870967741935484 %
Character:  0.1935483870967742 %
Character:  0.2 

Character:  3.296774193548387 %
Character:  3.303225806451613 %
Character:  3.3096774193548386 %
Character:  3.3161290322580643 %
Character:  3.3225806451612905 %
Character:  3.329032258064516 %
Character:  3.335483870967742 %
Character:  3.3419354838709676 %
Character:  3.3483870967741933 %
Character:  3.3548387096774195 %
Character:  3.361290322580645 %
Character:  3.367741935483871 %
Character:  3.3741935483870966 %
Character:  3.3806451612903228 %
Character:  3.3870967741935485 %
Character:  3.393548387096774 %
Character:  3.4 %
Character:  3.4064516129032256 %
Character:  3.412903225806452 %
Character:  3.4193548387096775 %
Character:  3.425806451612903 %
Character:  3.432258064516129 %
Character:  3.4387096774193546 %
Character:  3.445161290322581 %
Character:  3.4516129032258065 %
Character:  3.458064516129032 %
Character:  3.464516129032258 %
Character:  3.470967741935484 %
Character:  3.47741935483871 %
Character:  3.4838709677419355 %
Character:  3.490322580645161 %
Character

Character:  4.974193548387097 %
Character:  4.980645161290322 %
Character:  4.987096774193549 %
Character:  4.993548387096774 %
Character:  5.0 %
Character:  5.006451612903226 %
Character:  5.012903225806451 %
Character:  5.019354838709678 %
Character:  5.025806451612903 %
Character:  5.032258064516129 %
Character:  5.038709677419355 %
Character:  5.04516129032258 %
Character:  5.051612903225807 %
Character:  5.058064516129032 %
Character:  5.064516129032258 %
Character:  5.070967741935484 %
Character:  5.077419354838709 %
Character:  5.083870967741936 %
Character:  5.090322580645161 %
Character:  5.096774193548387 %
Character:  5.103225806451613 %
Character:  5.109677419354838 %
Character:  5.116129032258065 %
Character:  5.122580645161291 %
Character:  5.129032258064516 %
Character:  5.135483870967742 %
Character:  5.141935483870967 %
Character:  5.148387096774194 %
Character:  5.15483870967742 %
Character:  5.161290322580645 %
Character:  5.167741935483871 %
Character:  5.1741935483

Character:  6.664516129032258 %
Character:  6.670967741935484 %
Character:  6.67741935483871 %
Character:  6.683870967741935 %
Character:  6.690322580645161 %
Character:  6.696774193548387 %
Character:  6.703225806451613 %
Character:  6.709677419354839 %
Character:  6.716129032258064 %
Character:  6.72258064516129 %
Character:  6.7290322580645165 %
Character:  6.735483870967742 %
Character:  6.741935483870968 %
Character:  6.748387096774193 %
Character:  6.754838709677419 %
Character:  6.7612903225806456 %
Character:  6.767741935483871 %
Character:  6.774193548387097 %
Character:  6.780645161290322 %
Character:  6.787096774193548 %
Character:  6.7935483870967746 %
Character:  6.8 %
Character:  6.806451612903226 %
Character:  6.812903225806451 %
Character:  6.819354838709677 %
Character:  6.825806451612904 %
Character:  6.832258064516129 %
Character:  6.838709677419355 %
Character:  6.84516129032258 %
Character:  6.851612903225806 %
Character:  6.858064516129033 %
Character:  6.86451612

Character:  8.348387096774193 %
Character:  8.35483870967742 %
Character:  8.361290322580645 %
Character:  8.36774193548387 %
Character:  8.374193548387098 %
Character:  8.380645161290323 %
Character:  8.387096774193548 %
Character:  8.393548387096773 %
Character:  8.4 %
Character:  8.406451612903226 %
Character:  8.412903225806451 %
Character:  8.419354838709678 %
Character:  8.425806451612903 %
Character:  8.432258064516128 %
Character:  8.438709677419356 %
Character:  8.44516129032258 %
Character:  8.451612903225806 %
Character:  8.458064516129033 %
Character:  8.464516129032258 %
Character:  8.470967741935484 %
Character:  8.477419354838709 %
Character:  8.483870967741936 %
Character:  8.490322580645161 %
Character:  8.496774193548386 %
Character:  8.503225806451614 %
Character:  8.509677419354839 %
Character:  8.516129032258064 %
Character:  8.522580645161291 %
Character:  8.529032258064516 %
Character:  8.535483870967742 %
Character:  8.541935483870967 %
Character:  8.54838709677

...

Character:  91.81935483870967 %
Character:  91.8258064516129 %
Character:  91.83225806451613 %
Character:  91.83870967741936 %
Character:  91.84516129032258 %
Character:  91.85161290322581 %
Character:  91.85806451612903 %
Character:  91.86451612903225 %
Character:  91.87096774193549 %
Character:  91.87741935483871 %
Character:  91.88387096774194 %
Character:  91.89032258064516 %
Character:  91.89677419354838 %
Character:  91.90322580645162 %
Character:  91.90967741935484 %
Character:  91.91612903225807 %
Character:  91.92258064516129 %
Character:  91.92903225806451 %
Character:  91.93548387096774 %
Character:  91.94193548387096 %
Character:  91.9483870967742 %
Character:  91.95483870967742 %
Character:  91.96129032258065 %
Character:  91.96774193548387 %
Character:  91.97419354838709 %
Character:  91.98064516129033 %
Character:  91.98709677419355 %
Character:  91.99354838709678 %
Character:  92.0 %
Character:  92.00645161290322 %
Character:  92.01290322580645 %
Character:  92.01935483

Character:  93.50322580645161 %
Character:  93.50967741935484 %
Character:  93.51612903225806 %
Character:  93.52258064516128 %
Character:  93.52903225806452 %
Character:  93.53548387096774 %
Character:  93.54193548387097 %
Character:  93.54838709677419 %
Character:  93.55483870967743 %
Character:  93.56129032258065 %
Character:  93.56774193548387 %
Character:  93.5741935483871 %
Character:  93.58064516129032 %
Character:  93.58709677419355 %
Character:  93.59354838709677 %
Character:  93.6 %
Character:  93.60645161290323 %
Character:  93.61290322580645 %
Character:  93.61935483870968 %
Character:  93.6258064516129 %
Character:  93.63225806451612 %
Character:  93.63870967741936 %
Character:  93.64516129032258 %
Character:  93.65161290322581 %
Character:  93.65806451612903 %
Character:  93.66451612903226 %
Character:  93.67096774193548 %
Character:  93.6774193548387 %
Character:  93.68387096774194 %
Character:  93.69032258064516 %
Character:  93.69677419354839 %
Character:  93.703225806

Character:  95.18709677419355 %
Character:  95.19354838709677 %
Character:  95.2 %
Character:  95.20645161290322 %
Character:  95.21290322580646 %
Character:  95.21935483870968 %
Character:  95.2258064516129 %
Character:  95.23225806451613 %
Character:  95.23870967741935 %
Character:  95.24516129032259 %
Character:  95.2516129032258 %
Character:  95.25806451612904 %
Character:  95.26451612903226 %
Character:  95.27096774193548 %
Character:  95.27741935483871 %
Character:  95.28387096774193 %
Character:  95.29032258064517 %
Character:  95.29677419354839 %
Character:  95.30322580645161 %
Character:  95.30967741935484 %
Character:  95.31612903225806 %
Character:  95.3225806451613 %
Character:  95.32903225806452 %
Character:  95.33548387096774 %
Character:  95.34193548387097 %
Character:  95.34838709677419 %
Character:  95.35483870967742 %
Character:  95.36129032258064 %
Character:  95.36774193548388 %
Character:  95.3741935483871 %
Character:  95.38064516129032 %
Character:  95.3870967741

Character:  96.87096774193549 %
Character:  96.87741935483871 %
Character:  96.88387096774194 %
Character:  96.89032258064516 %
Character:  96.89677419354838 %
Character:  96.90322580645162 %
Character:  96.90967741935484 %
Character:  96.91612903225807 %
Character:  96.92258064516129 %
Character:  96.92903225806451 %
Character:  96.93548387096774 %
Character:  96.94193548387096 %
Character:  96.9483870967742 %
Character:  96.95483870967742 %
Character:  96.96129032258065 %
Character:  96.96774193548387 %
Character:  96.97419354838709 %
Character:  96.98064516129033 %
Character:  96.98709677419355 %
Character:  96.99354838709678 %
Character:  97.0 %
Character:  97.00645161290322 %
Character:  97.01290322580645 %
Character:  97.01935483870967 %
Character:  97.02580645161291 %
Character:  97.03225806451613 %
Character:  97.03870967741935 %
Character:  97.04516129032258 %
Character:  97.0516129032258 %
Character:  97.05806451612904 %
Character:  97.06451612903226 %
Character:  97.07096774

Character:  98.56129032258065 %
Character:  98.56774193548387 %
Character:  98.5741935483871 %
Character:  98.58064516129032 %
Character:  98.58709677419355 %
Character:  98.59354838709677 %
Character:  98.6 %
Character:  98.60645161290323 %
Character:  98.61290322580645 %
Character:  98.61935483870968 %
Character:  98.6258064516129 %
Character:  98.63225806451612 %
Character:  98.63870967741936 %
Character:  98.64516129032258 %
Character:  98.65161290322581 %
Character:  98.65806451612903 %
Character:  98.66451612903226 %
Character:  98.67096774193548 %
Character:  98.6774193548387 %
Character:  98.68387096774194 %
Character:  98.69032258064516 %
Character:  98.69677419354839 %
Character:  98.70322580645161 %
Character:  98.70967741935483 %
Character:  98.71612903225807 %
Character:  98.72258064516129 %
Character:  98.72903225806452 %
Character:  98.73548387096774 %
Character:  98.74193548387096 %
Character:  98.7483870967742 %
Character:  98.75483870967741 %
Character:  98.7612903225

In [15]:
print("count: ", count)
print("correct", correct)
print("ratio: ", correct/count)
print(difference)

count:  774800
correct 772277
ratio:  0.9967436757872999
defaultdict(<class 'int'>, {0: 772277, -1: 1281, 1: 978, 3: 264})
